In [1]:
import os
if 'PoC' not in os.listdir(os.curdir):
    %cd ../
import scraper.lemonde_scraper
import numpy as np
from scipy.sparse import find
feats = scraper.lemonde_scraper.loadFeaturesArticlesAsJson( "data/features")

/home/eric/GitGalaxy/news-scraper


# Preprocessing

## Select the data

For each article gather all the text I want to search

In [2]:
news_body = []
for feat in feats:
    news_body.append( feat['title'] + '\n' + 
                      feat['article_description'] + '\n' + 
                      feat['article_content']
                    )

## Transform the text to features vector

The TfIdf has been selected as it is an easy and fast way to have a search engine.

In [3]:
f = open("ranking/fr_stop_words.txt")
words = f.read()
stop_words = words.split('\n')

for i in range(len(stop_words)-1, -1, -1):
  if stop_words[i] == "" or stop_words[i][0] == "#":
    del stop_words[i]

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

stop_words=["le", "lambda"]
text_clf = Pipeline([('vect', CountVectorizer(stop_words=[""])),
                     ('tfidf', TfidfTransformer())])

In [5]:
news_body_tfidf = text_clf.fit_transform( news_body )
news_body_tfidf.shape

(116, 14106)

# About the data

# Ranking

In [6]:
dd = text_clf.steps[0][1].get_feature_names()

In [8]:
query = "facebook"
# TODO: lowercase query, remove 0 score
cumul_tfidf = np.zeros((116,1))

words_to_query = []
for w in query.split():
    if w in dd:
        words_to_query.append(dd.index(w))
    elif w in stop_words:
        print(w + " is a stop word.")
    else:
        print(w + " doesn't appear in any article.")
        
for w_index in words_to_query:
    article_index, _, tfidf_query = find(news_body_tfidf[:, w_index])
    for index, a_index in enumerate(article_index):
        cumul_tfidf[a_index] = cumul_tfidf[a_index] + tfidf_query[index]
        
sort_articles = cumul_tfidf.argsort(axis=0)[::-1].flatten() # Descending sorting
for i in sort_articles[0:10]:
    print(" * " + feats[i]['title'])
    print(" ----> Score: " + str(cumul_tfidf[i]) )

 * L’intégrité de Facebook « plus importante que les profits », selon Mark Zuckerberg
 ----> Score: [ 0.292967]
 * La numéro deux du gouvernement espagnol présentée à tort comme la fille d’un général franquiste
 ----> Score: [ 0.08814523]
 * Kevin Spacey renvoyé de « House of Cards »
 ----> Score: [ 0.04272676]
 * Sauvetage en Méditerranée : les corps de 23 personnes repêchés
 ----> Score: [ 0.03596742]
 * Menaces contre la chroniqueuse Nadia Daam : Europe 1 porte plainte
 ----> Score: [ 0.02410496]
 * Attaque à New York, Catalogne, harcèlement : les actualités à retenir cette semaine
 ----> Score: [ 0.02196305]
 * Donald Trump et la Russie, un an de liaisons dangereuses
 ----> Score: [ 0.01246234]
 * Petit manuel pour lutter contre les idées simplistes sur le harcèlement sexuel
 ----> Score: [ 0.00776834]
 * La stratégie d’Oslo pour réduire, voire éliminer, les voitures personnelles
 ----> Score: [ 0.]
 * Un missile tiré depuis le Yémen intercepté au-dessus de la capitale d’Arabie sao